# Assignment 2
## Digdarshan Kunwar, Eaindra Wun Pyae

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from numpy import genfromtxt
from skimage import io
from PIL import Image
plt.rcParams["figure.figsize"] = (25,5)